In [7]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))
tf.test.gpu_device_name()

[]


2024-03-09 11:29:02.616348: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


''